In [40]:
data_file_path = "/home/jovyan/work/finance_complaint/finance_artifact/data_preprocessing/20220906_130727/complaint_data"

In [41]:
from finance_complaint.entity.spark_manager import spark_session

In [56]:
spark_session

In [3]:
df = spark_session.read.parquet(data_file_path)

In [4]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder

In [5]:
ONE_HOT_FEATURES = ['company_response','consumer_consent_provided','submitted_via','timely']

NUMERICAL_FEATURE = ['diff_in_days']


FREQUENCY_ENCODER = ['company','issue','product','state','zip_code']

#TOEKNIZER = ['complaint_what_happened']


TARGET_FEATURE = ['consumer_disputed']

In [6]:
indexer = StringIndexer(inputCols=ONE_HOT_FEATURES,outputCols=[f"idx_{feature}" for feature in ONE_HOT_FEATURES])

In [7]:
from pyspark.ml.feature import Imputer


In [8]:
from pyspark.sql.functions import col

In [9]:
from pyspark.sql import DataFrame

In [10]:
from pyspark.sql.functions import desc

In [11]:
from typing import List

In [12]:
def get_top_category(dataframe:DataFrame,columns:List[str]):
    top_category=dict()
    for column in columns:
        top_cat = df.groupBy(column).count().sort(desc('count')).take(1)[0]
        print(top_cat)
        category = df.groupBy(column).count().sort(desc('count')).take(1)[0][column]
        
        top_category[column]=category

    return top_category



In [97]:
get_top_category(df,ONE_HOT_FEATURES)

Row(company_response='Closed with explanation', count=580728)
Row(consumer_consent_provided='N/A', count=470178)
Row(submitted_via='Web', count=524238)
Row(timely='Yes', count=748267)


{'company_response': 'Closed with explanation',
 'consumer_consent_provided': 'N/A',
 'submitted_via': 'Web',
 'timely': 'Yes'}

In [99]:
df.filter(col('zip_code').isNull()).count()

5660

In [14]:
for col in df.columns:
    setattr(df,col,col)

In [101]:
df = df.na.fill(get_top_category(df,ONE_HOT_FEATURES))

Row(company_response='Closed with explanation', count=580728)
Row(consumer_consent_provided='N/A', count=470178)
Row(submitted_via='Web', count=524238)
Row(timely='Yes', count=748267)


In [16]:
df.columns

['company',
 'company_response',
 'complaint_what_happened',
 'consumer_consent_provided',
 'consumer_disputed',
 'issue',
 'product',
 'state',
 'submitted_via',
 'timely',
 'zip_code',
 'diff_in_days']

In [28]:
.agg({"diff_in_days":"sum"}).show()

TypeError: 'str' object is not callable

In [30]:
df.printSchema()

root
 |-- company: string (nullable = true)
 |-- company_response: string (nullable = true)
 |-- complaint_what_happened: string (nullable = true)
 |-- consumer_consent_provided: string (nullable = true)
 |-- consumer_disputed: string (nullable = true)
 |-- issue: string (nullable = true)
 |-- product: string (nullable = true)
 |-- state: string (nullable = true)
 |-- submitted_via: string (nullable = true)
 |-- timely: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- diff_in_days: double (nullable = true)



In [39]:
df.filter(f"{df.diff_in_days} is  null").count()

769598

In [55]:
df.groupBy(['zip_code']).count().filter('zip_code is not null').sort(desc('count')).show()

+--------+-----+
|zip_code|count|
+--------+-----+
|   48382|  944|
|   33173|  691|
|   33071|  688|
|   30349|  598|
|   30331|  549|
|   30058|  528|
|   11375|  506|
|   92101|  500|
|   20744|  485|
|   20774|  476|
|   20002|  467|
|   90046|  459|
|   28269|  451|
|   33025|  442|
|   30043|  441|
|   30281|  436|
|   30039|  433|
|   32256|  432|
|   30294|  428|
|   75052|  418|
+--------+-----+
only showing top 20 rows



In [56]:
df = spark_session.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])
 
# Produces the following table:

In [58]:
df.show()

+---+----------------+-----+
| id|            text|label|
+---+----------------+-----+
|  0| a b c d e spark|  1.0|
|  1|             b d|  0.0|
|  2|     spark f g h|  1.0|
|  3|hadoop mapreduce|  0.0|
+---+----------------+-----+



In [62]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
 
# Define our transformation
def append_string(s, append_val=""):
    """
    If we see the word `spark` in s, append a string to the current string.
    """
    if s and 'spark' in s:
        return s + append_val
    return s
 
 
# Wrap the transformation as a UDF
append_udf = udf(lambda row: append_string(row, " hadoop"), StringType())
 
# Apply the UDF to our Dataset and create a resultant column called `appended_text`
df.withColumn(col("appended_text"), append_udf(col("text"))).show() 

TypeError: 'str' object is not callable

In [ ]:
import append_string  # this is the function we wrote above
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark import keyword_only 
 # Note: use pyspark.ml.util.keyword_only 
 # if Spark < 2.0 from pyspark.ml import Transformer 
 # #from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param, 
 # Params, TypeConverters from pyspark.ml.util import DefaultParamsReadable
 # , DefaultParamsWritable class StringAppender(Transformer,
 #  # Base class HasInputCol, # Sets up an inputCol
 #  parameter HasOutputCol, # Sets up an outputCol parameter DefaultParamsReadable,
 #  # Makes parameters readable from file DefaultParamsWritable 
 # # Makes parameters writable from file ): 
 # """ Custom Transformer wrapper class for append_string() """ 
 # # append_str is a value which we would like to be able to store state for,
 #  so we create a parameter. append_str = Param( Params._dummy(), "append_str", 
 # "Value we want to append with", typeConverter=TypeConverters.toString, # This 
 # will allow code to automatically try to convert to string )
 #  @keyword_only def __init__(self, inputCol=None, outputCol=None, append_str=None): 
 # """ Constructor: set values for all Param objects """ 
 # super().__init__() self._setDefault(append_str=None)
 #  kwargs = self._input_kwargs self.setParams(**kwargs)
 #  @keyword_only def setParams(self, inputCol=None, outputCol=None, append_str=None):
 #  kwargs = self._input_kwargs return self._set(**kwargs)
 #  def setAppendStr(self, new_append_str):
 #  return self.setParams(append_str=new_append_str) # Required if you use Spark >= 3.0
    def setInputCol(self, new_inputCol):
        return self.setParams(inputCol=new_inputCol)
  
    # Required if you use Spark >= 3.0
    def setOutputCol(self, new_outputCol):
        return self.setParams(outputCol=new_outputCol)
  
    def getAppendStr(self):
        return self.getOrDefault(self.append_str)
  
    def _transform(self, dataset):
        """
        This is the main member function which applies the transform to transform data from the `inputCol` to the `outputCol`
        """
        if not self.isSet("inputCol"):
            raise ValueError(
                "No input column set for the "
                "StringAppenderTransformer transformer."
            )
        input_column = dataset[self.getInputCol()]
        output_column = self.getOutputCol()
        append_str = self.getAppendStr()
        udf_func = lambda x: append_string(x, append_str)
        data_type = StringType()
         
        return dataset.withColumn(output_column,
                                  udf(udf_func, data_type)(input_column))

In [65]:
from pyspark.ml.base import Transformer

In [ ]:
class StandardScaler(Transformer):

    def __init__(self,*args,**kwargs) -> None:
        super().__init__(kwargs)
        

    